# Annotated 2D Molecular Visualization with RDKit


Package requirements:
---------------------
Python 3  
IOData  
RDKit  


Goal:
-----
Annotate and visualize a molecule with RDKit.


The process is as follows:
1. Save the melecule data as a PDB file thrugh IOData.
2. Load this file with RDKit and annotate the atoms with some scalar property from IOData.
3. Visualize the 2D drawing of the molecule.

Note: Due to RDKit not parsing correctly the .sdf and .mol2 files written with IOData, in this example an .pdb is used instead. Also, the molecule's SMILES code was required to assign bond orthers.

In [5]:
#
# Import required modules
#
from os import path
import numpy as np
from iodata import load_one, dump_one
from rdkit.Chem import MolFromPDBFile, MolFromSmiles, AddHs, AllChem, Draw


Bellow some auxiliar functions are defined to handle finding the initial molecule file and labeling the atoms with RDKit.

In [6]:
def find_datafile(fname):
    """Path finder helper function """
    # Construct absolute path and split by package name.
    # root = path.abspath(__file__).split("chemtools")[0]
    # See stackoverflow solutions for `__file__ does not exist in Jupyter Notebook` 
    root = os.getcwd().split("chemtools")[0]
    DATAPATH = path.join(root, "chemtools", "chemtools", "data", "examples", fname)
    return path.abspath(DATAPATH)


def rdkit_annotate_mol(fname, smiles, values):
    """Label atoms with a given scalar property in RDKit

    Parameters
    ----------
    fname : str
        Name of .pdb file used in RDKit to visualize the annotations.
    smiles : str
        SMILES code for the molecule used to assign the bond orders.
    values : list/np.ndarray
        A (N,) sequence of float values to be annotated.

    Returns
    -------
    RDKit molecule with annotations.
    """
    # Load molecule with RDKit
    # Assing bond orders using SMILES code
    mol = MolFromPDBFile(fname+'.pdb', sanitize=True, removeHs=False)
    AllChem.Compute2DCoords(mol)
    ref= MolFromSmiles(smiles)
    ref= AddHs(ref)
    mol = AllChem.AssignBondOrdersFromTemplate(ref,mol)
    
    # Label atoms by property
    for atom in mol.GetAtoms():
        idx = atom.GetIdx()
        atom.SetProp('atomNote',f"{values[idx]:.2}")
    
    return mol

def rdkit_moldraw_annotated(fname, smiles, esp, addstereo=True, addatomids=False):
    # Add labels to molecule
    mol = rdkit_annotate_mol(fname, smiles, esp)
    # Draw and save it
    d = Draw.rdMolDraw2D.MolDraw2DCairo(250, 200)
    d.drawOptions().addStereoAnnotation = addstereo
    d.drawOptions().addAtomIndices = addatomids
    d.DrawMolecule(mol)
    d.WriteDrawingText(fname+'.png')

Step 1.: Save the melocule as a PDB file
-------------------------------------------------

In [7]:
# 1. Load molecule data
fname = "dichloropyridine26_q+0"
smiles = 'C1=CC(=NC(=C1)Cl)Cl'
data = load_one(find_datafile(f"{fname}.fchk"))

# 2. Print a PDB file to be loaded by RDKit
dump_one(data, f"{fname}.pdb")

Steps 2. and 3.: Visualize it with RDKit
--------------------------------------------

In [8]:
# 3. Annotate the melecule with its scalar properties (e.g. atomic charges)
#    and save the visualization.
esp = data.atcharges["esp"]
rdkit_moldraw_annotated(fname, smiles, esp)